# Exploration 14

# 1) 데이터 준비와 전처리

Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [1]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head(300)

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
295,5,3728,2,978244568
296,5,229,3,978246528
297,5,6,2,978245916
298,5,3006,3,978245422


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [4]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre',] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head(10)

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [6]:
print(movies.values)
print(movies.values[0][0])
print(movies.values[0][1])

[[1 'Toy Story (1995)' "Animation|Children's|Comedy"]
 [2 'Jumanji (1995)' "Adventure|Children's|Fantasy"]
 [3 'Grumpier Old Men (1995)' 'Comedy|Romance']
 ...
 [3950 'Tigerland (2000)' 'Drama']
 [3951 'Two Family House (2000)' 'Drama']
 [3952 'Contender, The (2000)' 'Drama|Thriller']]
1
Toy Story (1995)


In [7]:
# movie_id 를 통해 title을 얻을 수 있도록 dictionary 생성
movie_dic = {}
for i in range(len(movies)):
    movie_dic[movies.values[i][0]] = movies.values[i][1]

In [8]:
# ratings에 새로운 열 추가
mv_title = []
for i in range(len(ratings)):
    mv_title.append(movie_dic[ratings.values[i][1]])

ratings['title'] = mv_title

In [9]:
ratings.head()

,user_id,movie_id,counts,timestamp,title
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,978302109,James and the Giant Peach (1996)
2,1,914,3,978301968,My Fair Lady (1964)
3,1,3408,4,978300275,Erin Brockovich (2000)
4,1,2355,5,978824291,"Bug's Life, A (1998)"


여기까지가 전처리입니다. 이후에는 이전 스텝에 소개했던 것과 동일한 방식으로 MF model을 구성하여 내가 좋아할 만한 영화를 추천해 볼 수 있습니다.

# 2) 분석해 봅시다.

- ratings에 있는 유니크한 영화 개수
- ratings에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

In [10]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [11]:
ratings['title'].nunique()

3628

In [12]:
# ratings에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [13]:
movie_dic[2858]

'American Beauty (1999)'

In [14]:
# 가장 인기 있는 영화 30개(인기순)
movie_count = ratings.groupby('title')['user_id'].count()
movie_count_30 = movie_count.sort_values(ascending=False).head(30)
movie_count_30

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

title이 보이지 않아서, 타이틀을 보이게끔 만들어 주었다.

In [15]:
# 위의 랭크의 movie_id가 아닌 title을 붙여주려 아래 코드를 하였지만
# 결국 ratings 의 열을 추가함으로써 아래 코드 불필요

# list_a = []
# list_b = []
# for i in range(len(movie_count_30)):
#     list_a.append(movie_count_30.values[i])
#     list_b.append(movies['title'][movie_count_30.index[i]-1])

# for i in range(len(movie_count_30)):
#     movie_count_30_wt = pd.Series(list_a, index = list_b)

# movie_count_30_wt

# 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍니다.

In [16]:
# 내가 좋아하는 영화 5개
my_favorite = ['Daeboo' , 'The Matrix' ,'Interstellar' ,'Leon' ,'Joker']

# 'Hong'이라는 user_id가 영화에 별점 5점씩.
my_movie = pd.DataFrame({'user_id': ['Hong']*5, 'title': my_favorite, 'counts':[5]*5})

if not ratings.isin({'user_id':['Hong']})['user_id'].any():  # user_id에 'Hong'이라는 데이터가 없다면
    ratings = ratings.append(my_movie)                      # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,counts,timestamp,title
1000203,6040,1090.0,3,956715518.0,Platoon (1986)
1000205,6040,1094.0,5,956704887.0,"Crying Game, The (1992)"
1000206,6040,562.0,5,956704746.0,Welcome to the Dollhouse (1995)
1000207,6040,1096.0,4,956715648.0,Sophie's Choice (1982)
1000208,6040,1097.0,4,956715569.0,E.T. the Extra-Terrestrial (1982)
0,Hong,NaN,5,NaN,Daeboo
1,Hong,NaN,5,NaN,The Matrix
2,Hong,NaN,5,NaN,Interstellar
3,Hong,NaN,5,NaN,Leon
4,Hong,NaN,5,NaN,Joker


In [17]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings['user_id'].unique()
title_unique = ratings['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [18]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['Hong'])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
print(title_to_idx['Joker'])

6039
3632


In [19]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# title_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = ratings['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(ratings):
    print('title column indexing OK!!')
    ratings['movie_id'] = temp_title_data
else:
    print('title column indexing Fail!!')

ratings

user_id column indexing OK!!
title column indexing OK!!


,user_id,movie_id,counts,timestamp,title
0,0,0,5,978300760.0,One Flew Over the Cuckoo's Nest (1975)
1,0,1,3,978302109.0,James and the Giant Peach (1996)
2,0,2,3,978301968.0,My Fair Lady (1964)
3,0,3,4,978300275.0,Erin Brockovich (2000)
4,0,4,5,978824291.0,"Bug's Life, A (1998)"
...,...,...,...,...,...
0,6039,3628,5,NaN,Daeboo
1,6039,3629,5,NaN,The Matrix
2,6039,3630,5,NaN,Interstellar
3,6039,3631,5,NaN,Leon


# 4) CSR matrix를 직접 만들어 봅시다.

In [20]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie_id = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings.counts, (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie_id))
csr_data

<6040x3633 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

# 5) als_model = AlternatingLeastSquares모델을 직접 구성하여 훈련시켜 봅시다.

In [21]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [97]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=200, regularization=0.01, use_gpu=False, iterations=20, dtype=np.float32)

1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
3. use_gpu : GPU를 사용할 것인지
4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

In [98]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3633x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [99]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/20 [00:00<?, ?it/s]

# 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [100]:
Hong, Joker = user_to_idx['Hong'], title_to_idx['Joker']
Hong_vector, Joker_vector = als_model.user_factors[Hong], als_model.item_factors[Joker]

print('슝=3')

슝=3


In [101]:
Hong_vector

array([-0.08537461, -0.11525011,  0.04184633,  0.08173399, -0.00789307,
       -0.01487048, -0.03378157,  0.03326793, -0.10368451,  0.09617941,
        0.0004333 , -0.09776039, -0.04150296, -0.10821933, -0.04119677,
        0.04559693, -0.05176141,  0.09579128, -0.11060125, -0.01929901,
        0.04458833,  0.00406039, -0.09182119, -0.059975  , -0.0338031 ,
        0.02200707,  0.03779413,  0.19063815,  0.03907008,  0.11574916,
       -0.07529662, -0.01197916,  0.09288327,  0.01425293, -0.00186133,
        0.07354088, -0.01053798,  0.03122463,  0.12905753, -0.02571228,
       -0.04529758,  0.01750619,  0.01618068,  0.00668929, -0.07933883,
       -0.0825027 ,  0.04808204, -0.0137686 ,  0.02453987,  0.11350194,
        0.1336938 ,  0.0216976 , -0.07830973, -0.05449667,  0.07451212,
        0.04298631,  0.00282762,  0.0060198 ,  0.17321438,  0.05070091,
        0.02217843, -0.1276341 , -0.0175557 , -0.10125414, -0.04588213,
       -0.0941361 ,  0.07202464,  0.09712252,  0.14419712,  0.08

In [102]:
Joker_vector

array([ 2.4401497e-03,  1.2331012e-03,  4.9678460e-03,  5.2804621e-03,
        2.8377080e-03,  4.6096388e-03,  3.0166269e-03,  3.8998898e-03,
        1.6083467e-03,  5.1354659e-03,  3.8123094e-03,  2.4202121e-03,
        3.0622720e-03,  8.1780285e-04,  3.5488559e-03,  4.4878586e-03,
        1.8512206e-03,  5.1624565e-03,  1.4656000e-03,  3.2149856e-03,
        3.7024159e-03,  3.4263944e-03,  9.7280176e-04,  2.2841888e-03,
        1.9965053e-03,  3.0310862e-03,  3.7288994e-03,  6.7250035e-03,
        3.5174079e-03,  6.0352767e-03,  2.5317373e-03,  3.6653879e-03,
        4.5249439e-03,  4.7527291e-03,  2.7951640e-03,  5.5978950e-03,
        3.9607906e-03,  4.5210449e-03,  5.7414197e-03,  2.8262192e-03,
        2.4493553e-03,  4.1213040e-03,  3.5179511e-03,  2.9041839e-03,
        3.0989779e-04,  1.0195842e-03,  5.0453153e-03,  2.1483651e-03,
        3.8492351e-03,  6.2156417e-03,  4.6127220e-03,  4.1880687e-03,
        1.9419889e-03,  2.5171754e-03,  4.2232806e-03,  3.4746351e-03,
      

In [103]:
# Hong과 Joker를 내적하는 코드
np.dot(Hong_vector, Joker_vector)

0.017990163

In [104]:
Leon = title_to_idx['Leon']
Leon_vector = als_model.item_factors[Leon]
np.dot(Hong_vector, Leon_vector)

0.018842101

# 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [105]:
favorite_movie = 'Joker'
movie_id = title_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(3628, 0.99999994),
 (3632, 0.99999994),
 (3629, 0.9999988),
 (3630, 0.99999666),
 (3631, 0.99994403),
 (3615, 0.99912083),
 (3531, 0.9984845),
 (3035, 0.99819624),
 (3088, 0.9977483),
 (3549, 0.9975962),
 (3591, 0.99753404),
 (3570, 0.9974574),
 (3420, 0.9972963),
 (3589, 0.9972597),
 (3053, 0.99717796)]

In [106]:
#title_to_idx 를 뒤집어, index로부터 title 이름을 얻는 dict를 생성합니다. 
idx_to_title = {v:k for k,v in title_to_idx.items()}
[idx_to_title[i[0]] for i in similar_movie]

['Daeboo',
 'Joker',
 'The Matrix',
 'Interstellar',
 'Leon',
 'Chain of Fools (2000)',
 'Aiqing wansui (1994)',
 'For Ever Mozart (1996)',
 'Open Season (1996)',
 'Make Them Die Slowly (Cannibal Ferox) (1980)',
 "Heaven's Burning (1997)",
 'Loser (1991)',
 'Jeanne and the Perfect Guy (Jeanne et le garçon formidable) (1998)',
 'Soft Toilet Seats (1999)',
 'Sunchaser, The (1996)']

In [108]:
# 함수로 만들기
def get_similar_title(movie_name: str):
    movie_id = title_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_title[i[0]] for i in similar_movie]
    return similar_movie

print("슝=3")

슝=3


In [112]:
get_similar_title('Loser (1991)')

['Loser (1991)',
 'The Matrix',
 'Daeboo',
 'Joker',
 'Interstellar',
 'Leon',
 'Chain of Fools (2000)',
 'Aiqing wansui (1994)',
 'Soft Toilet Seats (1999)',
 'Open Season (1996)']

# 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [113]:
user = user_to_idx['Hong']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(126, 0.029659528),
 (706, 0.028280737),
 (228, 0.026950791),
 (851, 0.026266616),
 (1523, 0.025484707),
 (2515, 0.024428526),
 (817, 0.024230957),
 (946, 0.023882747),
 (1553, 0.023631537),
 (2250, 0.02339926),
 (1135, 0.023101792),
 (1716, 0.022877868),
 (1143, 0.022647873),
 (1634, 0.022522487),
 (2336, 0.022416618),
 (3418, 0.022339216),
 (726, 0.022255087),
 (2987, 0.021940025),
 (832, 0.02185981),
 (2886, 0.021839507)]

In [114]:
[idx_to_title[i[0]] for i in movie_recommended]

['Shakespeare in Love (1998)',
 'L.A. Story (1991)',
 'X-Files: Fight the Future, The (1998)',
 'Iron Giant, The (1999)',
 'Flying Tigers (1942)',
 'Life and Times of Hank Greenberg, The (1998)',
 'Sneakers (1992)',
 'Parenthood (1989)',
 'Slipper and the Rose, The (1976)',
 'Blood on the Sun (1945)',
 'For All Mankind (1989)',
 "Angela's Ashes (1999)",
 'Do the Right Thing (1989)',
 'Everest (1998)',
 'Stalingrad (1993)',
 'I Love You, I Love You Not (1996)',
 'Blair Witch Project, The (1999)',
 'Grandfather, The (El Abuelo) (1998)',
 'Jakob the Liar (1999)',
 'New Age, The (1994)']

AlternatingLeastSquares 클래스에 구현된 explain 메서드를 사용하면 제가 기록을 남긴 데이터 중 이 추천에 기여한 정도를 확인할 수 있습니다.

In [129]:
Everest = title_to_idx['Everest (1998)']
explain = als_model.explain(user, csr_data, itemid=Everest)

이 method는 추천한 콘텐츠의 점수에 기여한 다른 콘텐츠와 기여도(합이 콘텐츠의 점수가 됩니다.)를 반환합니다. 

In [130]:
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('The Matrix', 0.004705807698421714),
 ('Leon', 0.004451247672594641),
 ('Daeboo', 0.004223749387509762),
 ('Joker', 0.004218785995173475),
 ('Interstellar', 0.004076339923804315)]

# 회고

## 이번 프로젝트에서 어려웠던 점
#### 이번 프로젝트에서 어려웠던 점은, 내적 수치를 높혀야 하나? 만약 그렇다면 어떻게 높히지? 였습니다.
- 우선, 학습 후에 정확도 같은 개념이 있지 않아, 수치적으로 볼 수 있는 부분이 유일하게 내적인데, 아티스트를 찾는 실습을 하였을 때는, 0.49 정도의 비교적 높은 내적 값이 나왔 었습니다. 하지만 제가 만든 영화 추천 내적같은경우에는, 심하면 e-15 정도나, 잘 나와도 지금 보시는 것 처럼 0.017 정도가 최선이었습니다. LMS에서 설명하기로는, 1이 나온다고 좋다고 볼 수는 없고, 이에 대한 검증이 필요하며, 또한 객관적이 평가가 어려우므로, 정답은 없다고 했는데, 이 부분이 이해하기 어려웠습니다.
- 또한, 우선 나름대로 최적을 값을 찾는다며 factors와, iterations값을 조정해 보았는데, 정말 놀랍게도.. 조정하는 값이 의미가 있나? 싶을정도로 값을 계속 바꾸어보아도, 혹은 같은 값으로 똑같이 학습하여도, 좋은 값이 나오지 못하였습니다.

## 프로젝트를 진행하면서 알아낸 점 혹은 아직 모호한 점
#### 추천시스템의 이론적인 부분에 대해 알 수 있었습니다.
- 완벽하게 이해하지는 못하였지만, 항상 유튜브나 넷플릭스나, 혹은 음악을 감상할 때에 어떤 방법으로 추천을 해주는지 내부적인 상황이 궁금하였는데, 이번 LMS와 실습을 통해 대략적인 부분을 알 수 있었습니다. 학생시절에 배운 수학의 내적을 이렇게 연관지을 수 있다는 점에서, 저로 하여금 다시 수학책을 펴게 만드는 재미난 순간이었습니다.
#### 정확도에 대해 모호합니다.
- 우선 위의 실습 같은경우에는, 개인적으로 모르는 영화도 많기에, 주관적으로 판단하고 싶었지만, 어떤 영화인지를 알 수 없기에 모호한 부분이 있었습니다. 또한, 내적의 값이 하이퍼 파라미터 값과는 무관하게 큰 폭의 차이를 갖고 있어서 어떤 의미가 있는지 역시 모호하였습니다.
- 다양한 내적 값을 가지고 다양하게 추천을 받아 보았는데, 전혀 다른 영화리스트를 보여줘서, 그리고 그 영화가 어떤 영화인지 알 수 없었기에, 정확도나 혹은 어떤 값을 어떻게 수정해야하는지에 대한 값이 잡히지 않아 다소 답답하였습니다.
- 하지만 한편으론.. 유튜브나 노래나 그런것도 마찬가지로 추천해줘도 마음에 들지 않는부분이 많기 때문에 지금 제가 구현한 추천시스템이 현재 제가 이용하고 있는 추천시스템과 유사하다고 느끼기도 하였습니다 ㅎㅎ

## 루브릭 평가 지표를 맞추기 위해 시도한 것들.
#### 사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.
- 처음 오픈한 ratings 데이터에는 title이 아닌 moive_id 만 기록되어 있었고, 두 번째 오픈한 movies 데이터에는 movie_id 와 title이 적혀있었습니다. 이를 이용하여, ratings 데이터에 movie_id에 걸맞는 title 열을 추가하였습니다. 이를 바탕으로 개수를 확인할 수 있었고, 혹시나 값이 틀리지 않았는지 재 검수도 하였습니다.

#### 시용자와 아이템 벡터 내적수치가 의미있게 형성되었다.
- 이 부분이 위에 말씀드린 가장 모호한 부분입니다. 수치적으로 의미가 있는지, 하이퍼 파라미터 값이 고정이어도 왜 내적 수치는 큰 폭을 변하는지에 대한 원인을 알 지 못하였습니다. 하지만 나름대로 다양한 값을 수정해 가며, 제 데이터 내에서 제가 할 수 있는 가장 높은 값을 찾아 추천을 받아 보았습니다.

#### MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다
- 어떤 점수가 유의미한 측정이 되었다고 말할 수 잌ㅅ는지 잘 모르겠습니다. 하지만 얼핏 보아도 유사도가 의미있는 측정이 되었다고 볼 수 없을것 같습니다. 우선 LMS 가장 마지막에, `filter_already_liked_items=True` 에 대한 버그가 있다고 쓰여있는데, 그 때문인지 이미 제가 본 영화 5개만 측정해 주었습니다. 이를 바꾸어 보려하였으나, 다른것을 건들 수가 없어 실패하였습니다.

## 만약에 루브릭 평가 관련 지표를 달성하지 못했을 때, 이유에 관한 추정
#### 내적 수치가 의미있게 형성되지 않았고, 아이템간 유사도, 기여도가 의미있게 측정되지 못한것이 달성하지 못하지 않았나 추정해 봅니다.
- 위에서 계속 설명하였듯, 하이퍼 파라미터를 같게하여도, 내적 수치가 큰폭을 변화하고, 최대치가 0.017정도 밖에 되지 않는 다는점에서, 제가 어떤 부분을 건드려야 이 값이향상하는지 알 수 없었기에 의미있는 값을 보여주지 못한것 같습니다. 그와 같은 이유로 인해 루브릭 평가 지표를 달성하지 못한 추정이 된다고 말하였습니다.

## 자기 다짐
- 이번 노드를 통하여 제가 실 생활에서 가장 밀접하게 경험하고 있다는 부분에 대해 내부적인 이론에 대해 알 수 있었고, 또한 수학공부를 할 때에 배웠던 내적 수치에 대한 활용을 알 수 있어서 한편으로는 너무 재미있었습니다. 하지만 이번 노드는 영화 관련 노드이고, 알지 못하는 영화들이 많았기에 추천해준 영화들에 대한 정확도는 물론 객관하하기 힘들지만, 제 자신이 알기가 쉽지 않았기에 어렵게 느껴졌습니다. 게다가 내적 값을 어느정도까지는 향상시키는 것이 맞다는 생각이 드는데, 어떤 식으로 어떤 방법으로 향상시킬 수 있을지에 대한 답을 아직 찾지 못한것이 마음이 걸립니다. 때문에 다음에도 추천관련 노드가 나왔을 때 이번 노드를 바탕으로 내적값의 의미와 유사도, 그리고 향상 시킬 수 있는 방법 등을 좀 더 공부해 볼 예정입니다.

감사합니다.